## RAG implementando método del paper (https://arxiv.org/html/2409.04701v2)

In [3]:
import os
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import openai
import nltk
from dotenv import load_dotenv
from PyPDF2 import PdfReader

# ------------------------------
# 1. CONFIGURACIÓN INICIAL
# ------------------------------

# Cargar variables de entorno (asegúrate de tener un archivo .env con OPENAI_API_KEY)
load_dotenv()

# Descargar los recursos necesarios de NLTK para tokenización
nltk.download('punkt')

# Configurar la API key de OpenAI
openai.api_key = os.getenv('OPENAI_API_KEY')

# Cargar el modelo de embeddings
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Parámetros de chunking a nivel local
max_chunk_tokens_ = 600
overlap_tokens_ = 80

# ------------------------------
# 2. FUNCIÓN PARA CARGAR PDFs
# ------------------------------

def load_pdf_texts(root_folder):
    """
    Recorre recursivamente el directorio 'root_folder' y extrae el texto de cada PDF.
    Se agrega metadata (nombre del bloque y del archivo) al inicio del texto.
    
    Retorna:
      - documents: lista de textos de cada PDF (con metadata incluida).
      - metadata: lista de diccionarios con información del bloque, nombre del archivo y ruta.
    """
    documents = []
    metadata = []
    
    for subdir, dirs, files in os.walk(root_folder):
        for file in files:
            if file.lower().endswith('.pdf'):
                file_path = os.path.join(subdir, file)
                try:
                    reader = PdfReader(file_path)
                    text = ""
                    for page in reader.pages:
                        page_text = page.extract_text()
                        if page_text:
                            text += page_text + "\n"
                    # Suponemos que el nombre del bloque es el nombre del directorio inmediato donde se encuentra el PDF
                    block_name = os.path.basename(subdir)
                    # Incluir metadata en el texto para que el LLM sepa el origen
                    document_text = f"[Bloque: {block_name} | PDF: {file}]\n{text}"
                    documents.append(document_text)
                    metadata.append({"block": block_name, "file": file, "path": file_path})
                except Exception as e:
                    print(f"Error al leer {file_path}: {e}")
    return documents, metadata

# Ruta de la carpeta que contiene los bloques (modifica esta ruta según tus necesidades)
pdf_root = "temario_opos"
documents, doc_metadata = load_pdf_texts(pdf_root)
print(f"Se han cargado {len(documents)} documentos desde '{pdf_root}'.")

# ------------------------------
# 3. CREACIÓN DEL ÍNDICE FAISS
# ------------------------------

# Generar embeddings para cada documento completo
document_embeddings = embedding_model.encode(documents, convert_to_numpy=True)

# Crear el índice FAISS usando la distancia L2
dimension = document_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(document_embeddings)

# ------------------------------
# 4. FUNCIÓN DE LATE CHUNKING
# ------------------------------

def late_chunking(text, max_chunk_tokens=600, overlap_tokens=80):
    """
    Fragmenta el texto en chunks de hasta 'max_chunk_tokens' tokens,
    agregando un solapamiento de 'overlap_tokens' tokens entre fragmentos.
    
    Se utiliza el tokenizador de oraciones de NLTK para no cortar oraciones a la mitad.
    """
    sentences = nltk.sent_tokenize(text)
    chunks = []
    current_chunk = ""
    current_tokens = 0

    for sentence in sentences:
        # Contamos tokens de forma aproximada separando por espacios
        sentence_tokens = sentence.split()
        num_tokens = len(sentence_tokens)
        
        # Si al agregar la oración se supera el límite, se guarda el chunk actual
        if current_tokens + num_tokens > max_chunk_tokens:
            chunks.append(current_chunk.strip())
            # Solapamiento: conservar los últimos 'overlap_tokens' tokens del chunk anterior
            current_chunk_tokens = current_chunk.split()
            current_chunk = " ".join(current_chunk_tokens[-overlap_tokens:]) if overlap_tokens > 0 else ""
            current_tokens = len(current_chunk.split())
        
        current_chunk += " " + sentence
        current_tokens += num_tokens

    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

# ------------------------------
# 5. FUNCIÓN PARA EXTRAER Y RANQUEAR CHUNKS RELEVANTES
# ------------------------------

def get_relevant_chunks(query_embedding, retrieved_docs, 
                        chunk_threshold=300, max_chunk_tokens=600, overlap_tokens=80):
    """
    Para cada documento recuperado, si es muy largo se aplica late chunking.
    Se vuelve a calcular el embedding para cada chunk y se obtiene una puntuación
    de similitud coseno con la consulta.
    
    Retorna una lista de diccionarios con keys: 'text', 'meta' y 'similarity'.
    """
    all_chunks = []
    # Para cada documento recuperado
    for doc_text, meta in retrieved_docs:
        # Si el documento es extenso, se fragmenta; de lo contrario, se usa completo
        if len(doc_text.split()) > chunk_threshold:
            chunks = late_chunking(doc_text, max_chunk_tokens=max_chunk_tokens, overlap_tokens=overlap_tokens)
        else:
            chunks = [doc_text]
        
        # Calcular la similitud de cada chunk con la consulta
        for chunk in chunks:
            chunk_embedding = embedding_model.encode(chunk, convert_to_numpy=True)
            # Asegurarse de trabajar con vectores unidimensionales
            q_emb = query_embedding.flatten()
            c_emb = chunk_embedding.flatten()
            # Calcular similitud coseno (se agrega un pequeño epsilon para evitar división por cero)
            similarity = np.dot(q_emb, c_emb) / (np.linalg.norm(q_emb) * np.linalg.norm(c_emb) + 1e-10)
            all_chunks.append({"text": chunk, "meta": meta, "similarity": similarity})
    
    # Ordenar todos los chunks de mayor a menor similitud
    all_chunks.sort(key=lambda x: x["similarity"], reverse=True)
    return all_chunks

# ------------------------------
# 6. FUNCIÓN PARA RESPONDER CONSULTAS
# ------------------------------

def answer_query(query, top_k=4, token_budget=15000):
    """
    Procesa la consulta del usuario:
      - Se genera el embedding de la consulta.
      - Se recuperan los top_k documentos completos usando FAISS.
      - Se aplica late chunking a cada documento recuperado y se recalcula la similitud
        de cada fragmento respecto a la consulta.
      - Se seleccionan los fragmentos con mayor similitud hasta no superar el presupuesto de tokens.
      - Se prepara el prompt combinando el contexto y la pregunta, y se consulta a la API de OpenAI.
    """
    # Generar embedding para la consulta (nota: encode espera una lista)
    query_embedding = embedding_model.encode([query], convert_to_numpy=True)
    
    # Recuperar los top_k documentos relevantes mediante FAISS
    distances, indices = index.search(query_embedding, top_k)
    retrieved_docs = []
    for i in indices[0]:
        doc_text = documents[i]
        meta = doc_metadata[i]
        retrieved_docs.append((doc_text, meta))
    
    # Aplicar late chunking y re-ranquear los fragments por similitud
    all_chunks = get_relevant_chunks(query_embedding, retrieved_docs)
    
    # Seleccionar fragmentos hasta no superar el presupuesto de tokens para el contexto
    selected_chunks = []
    total_tokens = 0
    for chunk_info in all_chunks:
        chunk_text = chunk_info["text"]
        # Conteo aproximado de tokens (por espacio)
        tokens = len(chunk_text.split())
        if total_tokens + tokens <= token_budget:
            selected_chunks.append(chunk_text)
            total_tokens += tokens
        else:
            break
    
    # Combinar los fragmentos seleccionados para formar el contexto
    context = "\n\n".join(selected_chunks)
    
    # Preparar el prompt para el modelo de lenguaje
    prompt = f"Contexto:\n{context}\n\nPregunta: {query}\nRespuesta:"
    
    # Llamar a la API de ChatCompletion (modelo: gpt-4o-mini)
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Eres un asistente educativo experto. Solo puedes responder a temas relacionados con las oposiciones."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.0,
    )
    
    return response.choices[0].message.content.strip()

# ------------------------------
# 7. BUCLE DE INTERACCIÓN CONTINUA
# ------------------------------

if __name__ == "__main__":
    print("Bienvenido al chatbot. Escribe 'salir' o 'exit' para terminar la conversación.")
    while True:
        consulta = input("Introduce tu consulta: ")
        if consulta.lower() in ["salir", "exit"]:
            print("Finalizando la conversación. ¡Hasta luego!")
            break
        respuesta = answer_query(consulta)
        print("\nRespuesta del chatbot:")
        print(respuesta)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/joseluis.fernandez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Se han cargado 67 documentos desde 'temario_opos'.
Bienvenido al chatbot. Escribe 'salir' o 'exit' para terminar la conversación.

Respuesta del chatbot:
Lo siento, pero no puedo responder preguntas que no estén relacionadas con oposiciones. Si tienes alguna consulta sobre temas de oposiciones, estaré encantado de ayudarte.

Respuesta del chatbot:
En el temario que has proporcionado, se pueden identificar al menos seis bloques principales. Estos bloques son:

1. **Bloque I**: La Constitución Española de 1978: estructura y contenido. La reforma de la Constitución.
2. **Bloque II**: Organización territorial (I): las Comunidades Autónomas. Los Estatutos de Autonomía.
3. **Bloque III**: Otras políticas públicas.
4. **Bloque IV**: Sistema sanitario: distribución de competencias, gestión y financiación.
5. **Bloque V**: Acción Exterior.
6. **Bloque VI**: Pagos a justificar. Anticipos de caja.

Si necesitas más detalles sobre cada bloque o su contenido, no dudes en preguntar.

Respuesta del c